In [ ]:
import csv
import boto3
import io

def lambda_handler(event, context):

    # Inicializa o cliente do S3
    s3 = boto3.client('s3')

    # Nome do bucket e do arquivo de entrada
    input_bucket = 'raw-data-plague'
    input_key = 'data_plague.csv'

    # Nome do bucket e do arquivo de saída
    output_bucket = 'refined-data-plague'
    output_key = 'data_refined_plague.csv'
    
    # Campos a serem mantidos
    columns_to_keep = [
        'continent',
        'location',
        'date',
        'total_cases',
        'new_cases',
        'total_deaths',
        'new_deaths',
        'reproduction_rate',
        'icu_patients',
        'hosp_patients',
        'total_tests',
        'new_tests',
        'tests_per_case',
        'total_vaccinations',
        'people_vaccinated',
        'people_fully_vaccinated',
        'population_density',
        'median_age',
        'aged_65_older',
        'aged_70_older',
        'gdp_per_capita',
        'extreme_poverty',
        'handwashing_facilities',
        'life_expectancy',
        'population'
    ]

    # Faz o download do arquivo CSV do S3
    response = s3.get_object(Bucket=input_bucket, Key=input_key)
    csv_content = response['Body'].read().decode('utf-8')

    # Lê o CSV e filtra as colunas desejadas
    csv_reader = csv.DictReader(io.StringIO(csv_content))
    filtered_rows = []
    
    for row in csv_reader:
        filtered_row = {field: row[field] for field in columns_to_keep if field in row}
        filtered_rows.append(filtered_row)

    # Nome das colunas filtradas
    fieldnames = columns_to_keep

    # Escreve o CSV filtrado para um buffer
    csv_buffer = io.StringIO()
    csv_writer = csv.DictWriter(csv_buffer, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(filtered_rows)

    # Faz o upload do CSV filtrado de volta para o S3
    s3.put_object(Bucket=output_bucket, Key=f'{output_key}', Body=csv_buffer.getvalue())
    
    return {
        'statusCode': 200,
        'body': f'File {output_key} has been saved in {output_bucket}'
    }
